In [20]:
%pip install -q pandas numpy plotly ipykernel jupyter


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from pathlib import Path

def find_root(targets=("data", "documents")):
    here = Path.cwd()
    for _ in range(5):
        if all((here / t).exists() for t in targets):
            return here
        here = here.parent
    raise FileNotFoundError(
        f"Could not locate a folder containing: {', '.join(targets)} starting at {Path.cwd()}"
    )

ROOT     = find_root()
DATA_DIR = ROOT / "data"
DOCS_DIR = ROOT / "documents"
DOCS_DIR.mkdir(parents=True, exist_ok=True)

CLEAN_CSV = DATA_DIR / "sms_members_cleaned.csv"
RAW1      = DATA_DIR / "sms_members_clinical_version.csv"
RAW2      = DATA_DIR / "mock_dsnp_members.csv"

print("ROOT     =", ROOT)
print("DATA_DIR =", DATA_DIR)
print("DOCS_DIR =", DOCS_DIR)
print("CLEAN_CSV exists? ", CLEAN_CSV.exists())


ROOT     = c:\Users\eluje\OneDrive\Documents\Lujean Important Docs\Portfolio-Mock Projects\lujean-portfolio1
DATA_DIR = c:\Users\eluje\OneDrive\Documents\Lujean Important Docs\Portfolio-Mock Projects\lujean-portfolio1\data
DOCS_DIR = c:\Users\eluje\OneDrive\Documents\Lujean Important Docs\Portfolio-Mock Projects\lujean-portfolio1\documents
CLEAN_CSV exists?  True


In [22]:
import pandas as pd
import numpy as np

if not CLEAN_CSV.exists():
    if RAW1.exists():
        df = pd.read_csv(RAW1)
        df = df.drop(columns=["notes", "system_flag"], errors="ignore")
        df = df.replace({"unknown": pd.NA})
        for col in ["last_hra_days", "response_history", "chronic_conditions", "age"]:
            df[col] = pd.to_numeric(df.get(col), errors="coerce")
        df = df.dropna(subset=["last_hra_days","response_history","chronic_conditions","age"])
        df.to_csv(CLEAN_CSV, index=False)
    elif RAW2.exists():
        df = pd.read_csv(RAW2)
        df.to_csv(CLEAN_CSV, index=False)
    else:
        raise FileNotFoundError(
            f"Missing {CLEAN_CSV.name} and no raw source found in {DATA_DIR}."
        )

df = pd.read_csv(CLEAN_CSV)
print("Loaded:", CLEAN_CSV)
df.head(3)


Loaded: c:\Users\eluje\OneDrive\Documents\Lujean Important Docs\Portfolio-Mock Projects\lujean-portfolio1\data\sms_members_cleaned.csv


,member_id,last_hra_days,response_history,chronic_conditions,age,dual_eligible,snp_member,zipcode
0,M00001,102.0,3.0,0.0,25.0,0,0,33613
1,M00002,348.0,1.0,1.0,39.0,0,0,33614
2,M00003,270.0,0.0,3.0,24.0,1,0,33614


In [23]:
# rename legacy -> new
if "last_hra_days" in df.columns and "hra_overdue_days" not in df.columns:
    df = df.rename(columns={"last_hra_days": "hra_overdue_days"})

# safe defaults for missing fields
df["managed_flag"]        = df["managed_flag"]        if "managed_flag"        in df.columns else 0
df["prior_hra_completed"] = df["prior_hra_completed"] if "prior_hra_completed" in df.columns else 0
df["high_risk_flag"]      = df["high_risk_flag"]      if "high_risk_flag"      in df.columns else (
    (df["chronic_conditions"] >= 3).astype(int) if "chronic_conditions" in df.columns else 0
)
df["recent_kickout_flag"] = df["recent_kickout_flag"] if "recent_kickout_flag" in df.columns else 0
df["er_visit_count"]      = df["er_visit_count"]      if "er_visit_count"      in df.columns else 0

# unmanaged only
df = df[df["managed_flag"] == 0].copy()

# score
def log_scale(days, max_days=365):
    return np.log1p(days) / np.log1p(max_days)

df["outreach_score"] = (
    log_scale(df["hra_overdue_days"]) * 30
    + (1 - (df["response_history"] / 5)) * 15
    + (df["chronic_conditions"] * 4 if "chronic_conditions" in df.columns else 0)
    + (df["age"] >= 75).astype(int) * 8
    + (df["prior_hra_completed"] * 5)
    + (df["high_risk_flag"] * 20)
    + (df["recent_kickout_flag"] * -10)
    + ((df["er_visit_count"] > 1).astype(int) * 10)
)


In [24]:
def assign_tier(score):
    if score >= 80:
        return "High"
    elif score >= 55:
        return "Medium"
    else:
        return "Low"

df["outreach_priority"] = df["outreach_score"].apply(assign_tier)

rng = np.random.default_rng(seed=42)
optin_p = {"High": 0.70, "Medium": 0.80, "Low": 0.90}
df["text_opt_in"] = df["outreach_priority"].map(lambda t: rng.binomial(n=1, p=optin_p[t]))

# back-compat column names for older cells
df["risk_tier"] = df.get("risk_tier", df["outreach_priority"])
df["score"]     = df.get("score",     df["outreach_score"])

import plotly.graph_objects as go


In [25]:
kpi_total      = len(df)
kpi_by_tier    = df["outreach_priority"].value_counts().sort_index()
kpi_optin_rate = df.groupby("outreach_priority")["text_opt_in"].mean().sort_index().round(3)

display({
    "Total unmanaged members": kpi_total,
    "Members by tier":         kpi_by_tier.to_dict(),
    "Opt-in rate by tier":     kpi_optin_rate.to_dict()
})


{'Total unmanaged members': 1898,
 'Members by tier': {'High': 85, 'Low': 1250, 'Medium': 563},
 'Opt-in rate by tier': {'High': 0.694, 'Low': 0.894, 'Medium': 0.796}}

In [26]:
import plotly.express as px

df_bar = df.assign(text_opt_in_label=np.where(df["text_opt_in"]==1, "Opt-in", "No Opt-in"))

fig_donut = px.pie(
    df, names="outreach_priority", hole=0.45,
    title="Outreach Priority Distribution (Unmanaged D-SNP)"
)
fig_bar = px.bar(
    df_bar, x="outreach_priority", color="text_opt_in_label", barmode="stack",
    title="Members by Priority Tier and Text Opt-in"
)
fig_hist = px.histogram(
    df, x="outreach_score", nbins=40, title="Outreach Score Distribution"
)

fig_donut.show(); fig_bar.show(); fig_hist.show()


In [27]:
HTML_PATH = DOCS_DIR / "sms_dashboard_plotly.html"

with open(HTML_PATH, "w", encoding="utf-8") as f:
    f.write("<!doctype html><meta charset='utf-8'><title>SMS Outreach Dashboard</title>")
    f.write("<h2>SMS Outreach Dashboard (Interactive)</h2>")
    f.write(fig_donut.to_html(full_html=False, include_plotlyjs="cdn"))
    f.write(fig_bar.to_html(full_html=False, include_plotlyjs=False))
    f.write(fig_hist.to_html(full_html=False, include_plotlyjs=False))

HTML_PATH


WindowsPath('c:/Users/eluje/OneDrive/Documents/Lujean Important Docs/Portfolio-Mock Projects/lujean-portfolio1/documents/sms_dashboard_plotly.html')

In [28]:
ranked_csv = DATA_DIR / "sms_outreach_ranked.csv"
df.sort_values("outreach_score", ascending=False).to_csv(ranked_csv, index=False)
ranked_csv


WindowsPath('c:/Users/eluje/OneDrive/Documents/Lujean Important Docs/Portfolio-Mock Projects/lujean-portfolio1/data/sms_outreach_ranked.csv')